In [ ]:
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta

# When compare the date between data sets, compare the following day delta 
days_delta_range = [0, -1, 1, -2, 2, -3, 3, -4, 4, -5, 5, -6, 6, -7, 7, -8, 8, -9, 9, -10, 10, -11, 11, -12, 12, -13, 13, -14, 14, -15, 15, -16, 16, -17, 17, -18, 18, -19, 19, -20, 20]

# Extract the whole volume 
def filter_dataframe_by_value(df, column_name, value_to_find):
    filtered_rows = df[df[column_name] == value_to_find]
    return filtered_rows

def load_csv(target):
    # Load the target dataset
    target_dataset = pd.read_csv(target['file_name']) #, index_col=target['date_column_name'])
    print(f"The dataset {target['file_name']} contains (row, column) = ")
    print(target_dataset.shape)
    return target_dataset

def save_csv(target, target_dataset):  
    # Drop all the index columns unnamed: 0 before saving 
    target_dataset.drop(target_dataset.filter(regex="Unname"),axis=1, inplace=True)
    # Save the data to its original file
    target_dataset.to_csv(target['file_name'])
    print(f"Saved to {target['file_name']}")

# Copy the matched column from the referral dataset to the target dataset
# The match key is the date. Note the date column might have different names
# The datetime must be close to each other between two datasets
def merge_dataframes_on_near_date(target_df, target, 
                                  referral_df, referral):

    #referral_df.head()
    
    # Sort the referral data set by Date, ascending 
    referral_df = referral_df.sort_values(referral['date_column_name'])

    for index1, row1 in target_df.iterrows():
        # Get the target datetime. 
        # Need to convert the string to a datetime object
        row1_datetime = datetime.strptime(str(row1[target['date_column_name']]),  target['datetime_format'])

        row1_near_date = []
        for i in days_delta_range:

            # Convert the day to int in order to compare with the referral_df.date, which is loaded as int by default
            next_day = row1_datetime + timedelta( days = i)
            #referral_next_day = int(next_day.strftime(referral['datetime_format']))
            referral_next_day = next_day.strftime(referral['datetime_format'])
            # If it is int ( not a string), convert to int as it is fast
            if  referral_df.dtypes[referral['date_column_name']] == 'int':
                referral_next_day = int(referral_next_day)
            row1_near_date.append( referral_next_day)

            # Find in the referral dataframe by date
            found_df = referral_df[referral_df[referral['date_column_name']] == referral_next_day]
            if (found_df.size > 0):
                print(f"Add data to the original {row1_datetime} <- {referral_next_day}  delta days {i}: ")
                for f in referral["data_fields_to_be_copied"]:
                    # Found the value in the first cell and assign to the target dataframe
                    # The target columne name might be different from the referral's. 
                    ### Hard code for now
                    target_column_name = f;
                    if target_column_name == "percent":
                        target_column_name = "fmc"

                    target_df.at[index1, target_column_name] = found_df.iloc[0][f]
                    print(f"{f} = {target_df.at[index1, target_column_name]}")
                
                # Finished fo this date
                break;

#
# Main program starts here
#
def merge_data_referral_to_target(target, referral): 

    # Load the target dataset
    target_dataset = load_csv(target)

    # Initial the referral dataset from a csv
    referral_dataset = load_csv(referral)

    # Check if the county exists in the referral dataset
    referral_count = referral_dataset['county'].str.contains(referral['referral_county_name']).sum()
    print(f"The referral county {referral['referral_county_name']} has {referral_count} rows of data in the {referral['file_name']}")  
    # If no data is found, exit the program
    if ( int(referral_count) <= 0):
        print("The county doesn't exist in the referral dataset. Exits the program.")
        return 
    
    # Create a new data frame containing only rows with the specific value
    referral_dataset_fips_only = filter_dataframe_by_value(referral_dataset, referral['fips_column_name'], referral['fips_code'])
    print(f"referral_dataset with fips {referral['fips_code']} = {referral_dataset_fips_only.shape}")

                
    # Add the new data from the referral dataset to the target dataset
    # merge_dataframes_on_match
    merge_dataframes_on_near_date(target_dataset, target, 
                                  referral_dataset_fips_only, referral)

    # Save the data to its original file
    save_csv(target, target_dataset)

def generate_csv_for_fips(target):

    fire_data = pd.read_csv("Datasets/aggregated_wildfire.csv")
    fire_data.head()
    # Create a new DataFrame containing only rows with the specific value
    firedata_fips = filter_dataframe_by_value(fire_data, 'FIPS', target["fips"])
    #new columns for merged features
    firedata_fips["tmax"] = 0
    firedata_fips["tmin"] = 0
    firedata_fips["tavg"] = 0
    firedata_fips["prcp"] = 0
    firedata_fips["aws"] = 0
    firedata_fips["fmc"] = 0
    save_csv(target, firedata_fips)

